In [ ]:
import os
import zipfile

if not os.path.isdir("source"):
   with zipfile.ZipFile("source.zip", 'r') as zip_ref:
      zip_ref.extractall(".")

if not os.path.isdir("data"):
   with zipfile.ZipFile("data.zip", 'r') as zip_ref:
      zip_ref.extractall(".")  

In [ ]:
import gzip
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image
import code
import tensorflow.python.platform
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as backend
import tensorflow_addons as tfa
import source.mask_to_submission as submission_maker
import source.constants as cst
import source.images as images
from sklearn.model_selection import KFold
import time

In [ ]:
def recall(y, predictions):
    true_positives = backend.sum(backend.round(backend.clip(y * predictions, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y, 0, 1)))
    recall = true_positives / (possible_positives + backend.epsilon())
    return recall

def precision(y, predictions):
    true_positives = backend.sum(backend.round(backend.clip(y * predictions, 0, 1)))
    predicted_positives = backend.sum(backend.round(backend.clip(predictions, 0, 1)))
    precision = true_positives / (predicted_positives + backend.epsilon())
    return precision

def f1_metric(y, predictions):
    pre = precision(y, predictions)
    rec = recall(y, predictions)
    return 2 * ((pre * rec) / (pre + rec + backend.epsilon()))

In [ ]:
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=1.0, gamma=2.0, reduction=tf.keras.losses.Reduction.AUTO, name='focal_loss'):
        super().__init__(reduction=reduction, name=name)
        self.alpha = alpha
        self.gamma = gamma

    def call(self, y_true, y_pred):
        y_true = tf.expand_dims(y_true, axis=-1)

        bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        pt = tf.math.exp(-bce_loss)

        return self.alpha * tf.math.pow(1-pt, self.gamma) * bce_loss -\
               (1-self.alpha) * tf.math.pow(pt, self.gamma) * tf.math.log(1-pt)


In [ ]:
def get_unet(alpha=1.0, gamma=2.0):
    inputs = layers.Input((200, 200, 3), name="input_layer")
    drop1 = layers.Dropout(cst.DROPOUT_PROBABILITY)(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(drop1)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2), (2, 2))(conv1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2), (2, 2))(conv2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D((2, 2), (2, 2))(conv3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D((2, 2), (2, 2))(conv4)
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
    up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    cropped6 = tf.image.resize_with_crop_or_pad(conv4, up6.shape[1], up6.shape[2])
    conc6 = layers.concatenate([up6, cropped6], axis=3)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conc6)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    cropped7 = tf.image.resize_with_crop_or_pad(conv3, up7.shape[1], up7.shape[2])
    conc7 = layers.concatenate([up7, cropped7], axis=3)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conc7)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    cropped8 = tf.image.resize_with_crop_or_pad(conv2, up8.shape[1], up8.shape[2])
    conc8 = layers.concatenate([up8, cropped8], axis=3)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conc8)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
    up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    cropped9 = tf.image.resize_with_crop_or_pad(conv1, up9.shape[1], up9.shape[2])
    conc9 = layers.concatenate([up9, cropped9], axis=3)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conc9)
    drop10 = layers.Dropout(cst.DROPOUT_PROBABILITY)(conv9)
    conv10 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(drop10)
    conv10 = layers.Conv2D(1, (1, 1), activation='sigmoid', name="output_layer")(conv10)

    unet = tf.keras.Model(inputs=[inputs], outputs=[conv10])
    unet.compile(optimizer='adam', loss=FocalLoss(alpha=alpha, gamma=gamma), metrics=[f1_metric, 'accuracy'])
    return unet

In [ ]:
def train_unet(unet, train_data, train_labels):

    output_shape = unet.get_layer("output_layer").output_shape
    margin = int((train_labels.shape[1] - output_shape[1]) / 2)
    right_margin = int(output_shape[1] + margin)
    train_labels = train_labels[:,margin:right_margin,margin:right_margin]

    unet.fit(train_data, train_labels, epochs=20, validation_split=0.0, batch_size=cst.BATCH_SIZE, callbacks=[], verbose=0)

    return mean_train, std_train

def evaluate_unet(unet, test_data, test_labels):
    output_shape = unet.get_layer("output_layer").output_shape
    margin = int((test_labels.shape[1] - output_shape[1]) / 2)
    right_margin = int(output_shape[1] + margin)
    test_labels = test_labels[:,margin:right_margin,margin:right_margin]

    return unet.evaluate(test_data, test_labels, batch_size=cst.BATCH_SIZE, verbose=0)

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits)

alphas = [0.6, 0.75, 0.9, 1.0]
gammas = [1.0, 2.0, 5.0]

ops = len(alphas)*len(gammas)*n_splits
results = np.zeros((len(alphas), len(gammas)), dtype=np.float64)

train_data_filename = cst.TRAIN_DIR + 'images/'
train_labels_filename = cst.TRAIN_DIR + 'groundtruth/' 

# Extract it into numpy arrays.
train_data, mean_train, std_train = images.load_training(train_data_filename, cst.TRAINING_SIZE)
train_labels = images.load_groundtruths(train_labels_filename, cst.TRAINING_SIZE)

print("DATA SHAPE " + str(train_data.shape))
print("TRAIN_LABELS SHAPE " + str(train_labels.shape))

In [ ]:
start = time.time()
ops_done=0

print("fold\talpha\tgamma\tf1\tcompl\telapsed\tleft")

for a, alpha in enumerate(alphas):
    for g, gamma in enumerate(gammas):
        intermediates = []
        cpt = 0
        for train, test in kf.split(train_data):
            X_train, X_test = train_data[train], train_data[test]
            Y_train, Y_test = train_labels[train], train_labels[test]

            unet = get_unet(alpha, gamma)
            train_unet(unet, X_train, Y_train)

            fbeta = evaluate_unet(unet, X_test, Y_test)[1]

            if fbeta > 1:
                fbeta = 0.

            intermediates.append(fbeta)

            ops_done += 1
            elapsed = time.time()-start
            left = ops*elapsed/ops_done - elapsed if ops_done != 0 else -1

            print("{}\t{:.2f}\t{:.2f}\t{:.4f}\t{:.2f}%\t{}s\t{}s".format(cpt, alpha, gamma, fbeta, ops_done/ops*100, int(elapsed), int(left)))
            cpt += 1
            del unet
        
        results[a][g] = np.median(intermediates)
        print()

In [ ]:
print("alpha\tgamma\tf1_score")
for a, alpha in enumerate(alphas):
    for g, gamma in enumerate(gammas):
        print("{:.2f}\t{:.2f}\t{:.4f}".format(alpha, gamma, results[a][g]))

a, g = np.unravel_index(np.argmax(results, axis=None), results.shape)

print("\nBest result with:")
print("alpha\tgamma\tf1_score")
print("{:.2f}\t{:.2f}\t{:.4f}".format(alphas[a], gammas[g], results[a][g]))